<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/complex-eigenvalues/60_linear_algebra_2/255_2DOF_Newton_Raphson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
import numpy as np
import numpy.linalg as nl
import numpy.random as nr
import numpy.testing as nt
import scipy.optimize as so



# Finding Eigenvalues of an Asymmetric Matrix using Newton Raphson Method<br>비대칭 행렬의 고유치를 뉴튼법으로 찾아보기



* A real square matrix $\mathbb{A}$ may have complex eigenvalues.<br>모든 원소가 실수인 정방행렬 $\mathbb{A}$도 비대칭이라면 그 고유치가 복소수일 수 있다.



* All eigenvalues of the matrix $\mathbb{A}$ are roots of its characteristic polynomial, what about trying the Newton-Raphson method?<br>행렬 $\mathbb{A}$의 고유치는 모두 그 특성방정식의 근이므로, 뉴튼랩슨법으로 찾아 보면 어떨까?




## Characteristic Equation



$$
    det(\mathbb{A}-\lambda \mathbb{I})=0
$$



## Example : 2DOF Mechanical Vibration System<br>사례: 2자유도 진동계



### State Space Form<br>상태공간 형태



* Equation of Motion<br>운동방정식



$$
      \mathbb{M}\frac{d^2}{dt^2}\mathbb{x}(t)
    + \mathbb{C}\frac{d}{dt}\mathbb{x}(t)
    + \mathbb{K}\mathbb{x}(t)=0
$$



* This is the fundamental equation governing forced and unforced vibration systems. Here:<br>자유 또는 강제 진동은 이 식을 따른다. 여기서:

|     | description<br>설명 |
|:----:|------|
| $\mathbb{M}$ | Mass matrix<br>질량 행렬 |
| $\mathbb{C}$ | Damping matrix<br>감쇠 행렬 |
| $\mathbb{K}$ | Stiffness matrix<br>강성 행렬 |
| $\mathbb{x}(t)$ | Displacement vector<br>변위 벡터 |



* Displacement Vector:<br>변위 벡터
    * Represents the displacement of each mass in the system over time.<br>시스템의 각 질량의 시간에 따른 변위를 나타냄.



$$
    \mathbb{x}(t) = \begin{bmatrix}
        x_1 (t) \\
        x_2 (t)
    \end{bmatrix}
$$



* Mass, Damping, and Stiffness Matricies<br>질량, 감쇠, 강성 행렬
    * These matrices define the specific properties of the 2DOF system. Notice how the arrangement of the elements reflects the connections between the masses, springs, and dampers.<br>이러한 행렬들이 해당 2자유도 시스템의 속성을 정의한다. 행렬의 원소들이 질량, 스프링, 댐퍼의 연결 상태를 어떻게 반영하고 있는지 자세히 살펴보기 바람.

$$
\begin{align}
    \mathbb{M} &= \begin{bmatrix}
        m_1 & 0 \\
        0 & m_2
    \end{bmatrix}\\
    \mathbb{C} &= \begin{bmatrix}
        c_1+c_2 & -c_2 \\
        -c_2 & c_2
    \end{bmatrix}\\
    \mathbb{K} &= \begin{bmatrix}
        k_1+k_2 & -k_2 \\
        -k_2 & k_2
    \end{bmatrix}
\end{align}
$$



#### Derivation<br>유도



* Rewriting the Equation of Motion:<br>운동방정식을 고쳐씀:
    * In these steps, the original equation of motion is rearranged through matrix manipulation to isolate the acceleration term.<br>아래와 같이 운동방정식을 재배열하여 가속도 항 만 등호의 왼쪽에 남김.



$$
      \mathbb{M}\frac{d^2}{dt^2}\mathbb{x}(t)
    = - \mathbb{C}\frac{d}{dt}\mathbb{x}(t)
    - \mathbb{K}\mathbb{x}(t)
$$



$$
      \frac{d^2}{dt^2}\mathbb{x}(t)
    = - \mathbb{M}^{-1}\mathbb{C}\frac{d}{dt}\mathbb{x}(t)
    - \mathbb{M}^{-1}\mathbb{K}\mathbb{x}(t)
$$



* Expanded Equation with Matrix Coefficients:<br>행렬 계수로 방정식을 표시
    * Here, the matrix-vector equations are written out explicitly, showing how each element corresponds to forces arising from the spring and damper connections between the masses.<br>행렬 벡터 방정식을 명시적으로 표시하여 각 요소가 질량 사이의 스프링 댐퍼 연결 상태로 부터 나타나는 힘을 보임.



$$
    \begin{align}
        \begin{bmatrix}
            \frac{d^2}{dt^2}x_1 (t) \\
            \frac{d^2}{dt^2}x_2 (t)
        \end{bmatrix}
        &= 
        \begin{bmatrix}
            \frac{-c_1-c_2}{m_1} & \frac{c_2}{m_1} \\
            \frac{c_2}{m_2} & \frac{-c_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            \frac{d}{dt}x_1 (t) \\
            \frac{d}{dt}x_2 (t)
        \end{bmatrix}
        +
        \begin{bmatrix}
            \frac{-k_1-k_2}{m_1} & \frac{k_2}{m_1} \\
            \frac{k_2}{m_2} & \frac{-k_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            x_1 (t) \\
            x_2 (t)
        \end{bmatrix} \\
        &= 
        \begin{bmatrix}
            \frac{-k_1-k_2}{m_1} & \frac{k_2}{m_1} \\
            \frac{k_2}{m_2} & \frac{-k_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            x_1 (t) \\
            x_2 (t)
        \end{bmatrix}
        +
        \begin{bmatrix}
            \frac{-c_1-c_2}{m_1} & \frac{c_2}{m_1} \\
            \frac{c_2}{m_2} & \frac{-c_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            \frac{d}{dt}x_1 (t) \\
            \frac{d}{dt}x_2 (t)
        \end{bmatrix}
    \end{align}
$$



* State Space Representation:<br>상태 공간 표현식:
    * The system is converted into state space form, where the state vector q(t) includes both displacements and their corresponding velocities. This format is convenient for analysis and  control purposes.<br>시스템을 상태공간 형태로 변형. 상태 벡터 $\mathbb{q}(t)$ 는 두 질량의 변위와 해당 속도를 포함. 이 형태는 해석 제어 목적으로 편리함.



$$
    \mathbb{q}(t) = \begin{bmatrix}
        q_1 (t) \\
        q_2 (t) \\
        q_3 (t) \\
        q_4 (t)
    \end{bmatrix} = \begin{bmatrix}
        x_1 (t) \\
        x_2 (t) \\
        \frac{d}{dt}x_1 (t) \\
        \frac{d}{dt}x_2 (t)
    \end{bmatrix}
$$



$$
    \frac{d}{dt}\mathbb{q}(t) 
    = \begin{bmatrix}
        \frac{d}{dt}q_1 (t) \\
        \frac{d}{dt}q_2 (t) \\
        \frac{d}{dt}q_3 (t) \\
        \frac{d}{dt}q_4 (t)
    \end{bmatrix}
    = \frac{d}{dt}\begin{bmatrix}
        x_1 (t) \\
        x_2 (t) \\
        \frac{d}{dt}x_1 (t) \\
        \frac{d}{dt}x_2 (t)
    \end{bmatrix}
    = \begin{bmatrix}
        \frac{d}{dt}x_1 (t) \\
        \frac{d}{dt}x_2 (t) \\
        \frac{d^2}{dt^2}x_1 (t) \\
        \frac{d^2}{dt^2}x_2 (t)
    \end{bmatrix} = \begin{bmatrix}
        q_3 (t) \\
        q_4 (t) \\
        \begin{bmatrix}
            \frac{-k_1-k_2}{m_1} & \frac{k_2}{m_1} \\
            \frac{k_2}{m_2} & \frac{-k_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            q_1 (t) \\
            q_2 (t)
        \end{bmatrix}
        +
        \begin{bmatrix}
            \frac{-c_1-c_2}{m_1} & \frac{c_2}{m_1} \\
            \frac{c_2}{m_2} & \frac{-c_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            q_3 (t) \\
            q_4 (t)
        \end{bmatrix}
    \end{bmatrix}
$$



* State Space Equation<br>상태공간방정식



$$
    \frac{d}{dt}\mathbb{q}(t) 
    = \begin{bmatrix}
        \frac{d}{dt}x_1 (t) \\
        \frac{d}{dt}x_2 (t) \\
        \frac{d^2}{dt^2}x_1 (t) \\
        \frac{d^2}{dt^2}x_2 (t)
    \end{bmatrix} = \begin{bmatrix}
        \begin{bmatrix}
            0 & 0 \\
            0 & 0
        \end{bmatrix}
        \begin{bmatrix}
            q_1 (t) \\
            q_2 (t)
        \end{bmatrix}
        +
        \begin{bmatrix}
            1 & 0 \\
            0 & 1
        \end{bmatrix}
        \begin{bmatrix}
            q_3 (t) \\
            q_4 (t)
        \end{bmatrix} \\
        \begin{bmatrix}
            \frac{-k_1-k_2}{m_1} & \frac{k_2}{m_1} \\
            \frac{k_2}{m_2} & \frac{-k_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            q_1 (t) \\
            q_2 (t)
        \end{bmatrix}
        +
        \begin{bmatrix}
            \frac{-c_1-c_2}{m_1} & \frac{c_2}{m_1} \\
            \frac{c_2}{m_2} & \frac{-c_2}{m_2}
        \end{bmatrix}
        \begin{bmatrix}
            q_3 (t) \\
            q_4 (t)
        \end{bmatrix}
    \end{bmatrix}
$$



* This represents the final first-order differential equation in matrix form. The matrix on the right-hand side is the system matrix, which encapsulates all the information about the masses, springs, and dampers.<br>이 형태가 최종 1차 상미분 방정식을 행렬 형태로 나타냄. 등호 오른쪽의 행렬이 시스템 행렬로, 질량, 스프링, 댐퍼의 모든 정보를 담고 있다.



$$
    \frac{d}{dt}\mathbb{q}(t) 
    = \begin{bmatrix}
        \mathbb{0} & \mathbb{I} \\
        -\mathbb{M}^{-1}\mathbb{K} & -\mathbb{M}^{-1}\mathbb{C}
    \end{bmatrix}
    \mathbb{q}
$$



### System Matrix $\mathbb{A}$<br>시스템 행렬 $\mathbb{A}$



* Upper part of $\mathbb{A}$<br>$\mathbb{A}$ 행렬의 윗부분



In [ ]:
ndof = 2

mA00 = np.zeros((ndof, ndof))
mA01 = np.identity(2)
mA0 = np.hstack((mA00, mA01))



* Lumped parameters<br>시스템 매개변수



In [ ]:
m1_kg = nr.uniform(0.1, 1)
m2_kg = m1_kg# nr.uniform(0.1, 1)

c1_Npmps = nr.uniform(0.1, 5)
c2_Npmps = c1_Npmps# nr.uniform(0.1, 5)

k1_Npm = nr.uniform(10, 100)
k2_Npm = k1_Npm # nr.uniform(10, 100)



* $\mathbb{M}^{-1}$



In [ ]:
mMinv = np.diag(((1.0/m1_kg), (1.0/m2_kg)))



* $\mathbb{C}$



In [ ]:
mC = np.array(
    [
        [(c1_Npmps+c2_Npmps), -c2_Npmps],
        [-c2_Npmps, c2_Npmps],
    ]
)



* $\mathbb{K}$



In [ ]:
mK = np.array(
    [
        [(k1_Npm+k2_Npm), -k2_Npm],
        [-k2_Npm, k2_Npm],
    ]
)



* Lower part of $\mathbb{A}$<br>$\mathbb{A}$ 행렬의 아랫부분



In [ ]:
mA10 = -mMinv @ mK
mA11 = -mMinv @ mC
mA1 = np.hstack((mA10, mA11))



In [ ]:
mA = np.vstack((
    mA0,
    mA1
))



* $\mathbb{A}$



In [ ]:
mA



### Characteristic Polynomial<br>특성 다항식



In [ ]:
import sympy as sym



In [ ]:
mAsym = sym.Matrix(mA)



In [ ]:
p_sym = mAsym.charpoly()
p_sym



In [ ]:
func_coefs = p_sym.all_coeffs()
func_coefs



In [ ]:
dfunc_coefs = p_sym.diff().all_coeffs()
dfunc_coefs



In [ ]:
func_coefs.reverse()
func = np.polynomial.Polynomial(np.array(func_coefs).astype(float))

dfunc_coefs.reverse()
dfunc = np.polynomial.Polynomial(np.array(dfunc_coefs).astype(float))



Make sure both polynomials are equal.<br>sympy 다항식과 numpy 다항식이 같은지 확인



In [ ]:
nt.assert_almost_equal(func(1), p_sym.eval(1))



### Newton Raphson Method<br>뉴튼 랩슨법



In [ ]:
import itertools
import os


eigenvalues = []

x0_real_array = np.linspace(-100, 100)

for x0_real, m, k in itertools.product(
        x0_real_array,
        (m1_kg, m2_kg),
        (k1_Npm, k2_Npm, 0.0),
    ):
    omega = (-k / m) ** 0.5
    x0 = x0_real + omega

    assert np.isfinite(func(x0))
    assert np.isfinite(dfunc(x0))
    try:
        e_new = so.newton(func, x0, fprime=dfunc)
        if all(map(lambda x: not np.isclose(x, e_new), eigenvalues)):
            eigenvalues.append(e_new)
        e_new_conj = e_new.conj()
        if all(map(lambda x: not np.isclose(x, e_new_conj), eigenvalues)):
            eigenvalues.append(e_new_conj)

    except RuntimeError as e:
        pass

print(eigenvalues)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

